In [1]:
import numpy as np
import pandas as pd
import json
import s3fs
import os
from io import BytesIO
os.environ["AWS_ACCESS_KEY_ID"] = 'ELTTPH6RVEAFN06SQ9ZG'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'AHz2feyFi8laSq8ecF2SrNPAguQtQxI197bn5dce'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJFTFRUUEg2UlZFQUZOMDZTUTlaRyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU0NjM2Mzg4LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImFudG9pbmUubGVzYXV2YWdlQGFncmljdWx0dXJlLmdvdXYuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU1NTAxMTMwLCJmYW1pbHlfbmFtZSI6IkxFU0FVVkFHRSIsImdpdmVuX25hbWUiOiJBbnRvaW5lIiwiZ3JvdXBzIjpbIlVTRVJfT05ZWElBIl0sImlhdCI6MTc1NDg5NjMzMCwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDpkNTUxYzZjZS04MjJiLTA3NDAtNjdkNi01ZTNkMjA4YTlkMzgiLCJsb2NhbGUiOiJmciIsIm5hbWUiOiJBbnRvaW5lIExFU0FVVkFHRSIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhbnRvaW5lbGVzYXV2YWdlIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzaWQiOiJkNzdiZWZlMC0xNTJiLTRlNTctODgxZi02M2I1YzAxNTZlODAiLCJzdWIiOiJlOWY4NjIyYy01MzQ2LTRjOGUtOWZmYi1jY2MwMzZjY2ZjZjciLCJ0eXAiOiJCZWFyZXIifQ.4GyXkD8zcg-q16M1ZC34z2jKYQFTeq4KsD6KJsKAOFkYbNo1Gb81DdsuV2u3MGORlwZHgo6Aj-A2F1KMxlHC7A'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'

import numpy as np
import pandas as pd
import json
import s3fs
import os

import numpy as np
import pandas as pd
import json
import s3fs
import os
import torch

def extract_instances_properties_corrected_robust(data, tile_id):
    """Version robuste qui gère les tenseurs PyTorch"""
    
    instances = []
    
    try:
        # Vérifier la présence des clés essentielles
        required_keys = ['instance_masks', 'instance_boxes', 'semantic']
        missing_keys = [key for key in required_keys if key not in data]
        if missing_keys:
            print(f"    ⚠️ Clés manquantes: {missing_keys}")
            return instances
        
        instance_masks = data['instance_masks']
        instance_boxes = data['instance_boxes']
        semantic_preds = data['semantic']
        
        # Vérifier qu'il y a des instances
        if len(instance_masks) == 0:
            print(f"    ⚠️ Aucun masque d'instance trouvé")
            return instances
        
        # Gérer confidence qui peut être float ou array
        if isinstance(data['confidence'], (int, float)):
            confidences = [data['confidence']] * len(instance_masks)
        elif isinstance(data['confidence'], np.ndarray):
            confidences = data['confidence'].flatten()
        elif hasattr(data['confidence'], 'cpu'):  # torch.Tensor
            confidences = data['confidence'].cpu().numpy().flatten()
        else:
            confidences = [0.5] * len(instance_masks)
        
        # Gérer centerness (optionnel)
        if 'centerness' in data and isinstance(data['centerness'], np.ndarray):
            centerness_scores = data['centerness'].flatten()
        elif 'centerness' in data and hasattr(data['centerness'], 'cpu'):
            centerness_scores = data['centerness'].cpu().numpy().flatten()
        else:
            centerness_scores = [0.5] * len(instance_masks)
        
        # Gérer size (optionnel)
        if 'size' in data and isinstance(data['size'], np.ndarray):
            sizes = data['size']
        elif 'size' in data and hasattr(data['size'], 'cpu'):
            sizes = data['size'].cpu().numpy()
        else:
            sizes = None
        
        # Convertir instance_boxes si c'est un tensor
        if hasattr(instance_boxes, 'cpu'):
            instance_boxes = instance_boxes.cpu().numpy()
        
        # Convertir semantic_preds si c'est un tensor
        if hasattr(semantic_preds, 'cpu'):
            semantic_preds = semantic_preds.cpu().numpy()
        
        print(f"    📊 Traitement de {len(instance_masks)} instances...")
        
        valid_instances = 0
        for i, (mask, bbox) in enumerate(zip(instance_masks, instance_boxes)):
            try:
                # Convertir le masque torch.Tensor en numpy
                if hasattr(mask, 'cpu'):
                    mask = mask.cpu().numpy()
                elif not isinstance(mask, np.ndarray):
                    print(f"    ⚠️ Instance {i}: type de masque non supporté ({type(mask)})")
                    continue
                
                # Vérifier que le masque n'est pas vide
                if mask.sum() == 0:
                    continue
                
                # Calculer l'aire en pixels
                area_pixels = int(mask.sum())
                
                # Conversion approximative en hectares
                area_ha = area_pixels * 0.01
                
                # Classe prédite
                if i < len(semantic_preds):
                    class_probs = semantic_preds[i]
                    if isinstance(class_probs, np.ndarray) and len(class_probs) > 0:
                        class_id = int(np.argmax(class_probs))
                        class_confidence = float(np.max(class_probs))
                    else:
                        class_id = 0
                        class_confidence = 0.5
                else:
                    class_id = 0
                    class_confidence = 0.5
                
                # Confidence globale
                global_confidence = float(confidences[i]) if i < len(confidences) else 0.5
                
                # Centerness
                centerness = float(centerness_scores[i]) if i < len(centerness_scores) else 0.5
                
                # Informations géométriques
                y1, x1, y2, x2 = bbox
                centroid_x = float((x1 + x2) / 2)
                centroid_y = float((y1 + y2) / 2)
                width = float(x2 - x1)
                height = float(y2 - y1)
                
                # Size prédite si disponible
                if sizes is not None and i < len(sizes):
                    pred_width, pred_height = sizes[i]
                else:
                    pred_width, pred_height = width, height
                
                instance = {
                    'tile_id': tile_id,
                    'instance_id': i,
                    'class_id': class_id,
                    'confidence': global_confidence,
                    'class_confidence': class_confidence,
                    'centerness': centerness,
                    'area_pixels': area_pixels,
                    'area_ha': area_ha,
                    'centroid_x': centroid_x,
                    'centroid_y': centroid_y,
                    'bbox_x1': float(x1),
                    'bbox_y1': float(y1),
                    'bbox_x2': float(x2),
                    'bbox_y2': float(y2),
                    'width': width,
                    'height': height,
                    'pred_width': float(pred_width),
                    'pred_height': float(pred_height)
                }
                
                instances.append(instance)
                valid_instances += 1
                    
            except Exception as e:
                print(f"    ❌ Erreur instance {i}: {e}")
                continue
        
        print(f"    ✅ {valid_instances} instances valides extraites sur {len(instance_masks)}")
                
    except Exception as e:
        print(f"    ❌ Erreur globale extraction: {e}")
    
    return instances

def process_all_files_safe_s3fs_pytorch():
    """Version corrigée pour gérer les tenseurs PyTorch"""
    
    bucket_name = os.environ.get('AWS_S3_BUCKET', 'antoinelesauvage')
    fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])
    
    try:
        preds_path = f"{bucket_name}/vergers-france/preds/"
        
        try:
            all_files = fs.glob(f"{preds_path}*_best_fold_*.npy")
        except FileNotFoundError:
            print("❌ Dossier preds/ non trouvé ou vide!")
            return pd.DataFrame(), {}
        
        if not all_files:
            print("❌ Aucun fichier *_best_fold_*.npy trouvé!")
            return pd.DataFrame(), {}
        
        pred_files = [f"s3://{f}" for f in all_files]
        
        all_instances = []
        processing_summary = {}
        successful_files = 0
        error_files = 0
        
        print(f"🎯 Traitement de {len(pred_files)} fichiers depuis S3...")
        
        # Traitement complet de tous les fichiers
        for i, s3_path in enumerate(pred_files):
            try:
                filename = os.path.basename(s3_path)
                tile_id = filename.split('_')[0]
                
                # Afficher le progrès tous les 1000 fichiers
                if i % 1000 == 0 or i < 10:  # Afficher les 10 premiers + tous les 1000
                    print(f"\n📍 [{i+1}/{len(pred_files)}] {filename}")
                
                # Lire le fichier
                with fs.open(s3_path, 'rb') as f:
                    data = np.load(f, allow_pickle=True).item()
                
                # Traiter les données
                instances = extract_instances_properties_corrected_robust(data, tile_id)
                
                if instances:
                    all_instances.extend(instances)
                    processing_summary[tile_id] = {
                        'n_instances': len(instances),
                        'total_area_ha': sum(inst['area_ha'] for inst in instances),
                        'avg_confidence': sum(inst['confidence'] for inst in instances) / len(instances),
                        'classes': list(set(inst['class_id'] for inst in instances))
                    }
                    if i % 1000 == 0 or i < 10:
                        print(f"  ✅ {len(instances)} instances extraites")
                    successful_files += 1
                else:
                    processing_summary[tile_id] = {
                        'n_instances': 0,
                        'error': 'Aucune instance extraite'
                    }
                    error_files += 1
                    
            except Exception as e:
                tile_id = filename if 'filename' in locals() else f"file_{i}"
                processing_summary[tile_id] = {
                    'n_instances': 0,
                    'error': str(e)
                }
                if i % 1000 == 0 or i < 10:
                    print(f"  ❌ Erreur: {e}")
                error_files += 1
            
            # Afficher un résumé intermédiaire tous les 5000 fichiers
            if (i + 1) % 5000 == 0:
                current_instances = len(all_instances)
                print(f"\n📊 Résumé intermédiaire après {i+1} fichiers:")
                print(f"  Instances trouvées: {current_instances}")
                print(f"  Fichiers réussis: {successful_files}")
                print(f"  Fichiers erreurs: {error_files}")
        
        # Résumé final
        print(f"\n📊 RÉSUMÉ FINAL:")
        print(f"  Total instances: {len(all_instances)}")
        print(f"  Fichiers traités avec succès: {successful_files}")
        print(f"  Fichiers avec erreurs: {error_files}")
        print(f"  Total fichiers: {len(pred_files)}")
        
        # Créer DataFrame même si vide
        if all_instances:
            df = pd.DataFrame(all_instances)
            print(f"\n  Classes trouvées: {sorted(df['class_id'].unique())}")
            print(f"  Surface totale: {df['area_ha'].sum():.2f} ha")
            print(f"  Confiance moyenne: {df['confidence'].mean():.3f}")
        else:
            df = pd.DataFrame()
            print("\n  ⚠️ Aucune instance trouvée dans tous les fichiers")
        
        # Sauvegarder
        results_path = f"s3://{bucket_name}/vergers-france/results/"
        if not fs.exists(results_path):
            fs.makedirs(results_path)
        
        csv_path = f"{results_path}all_instances_complete.csv"
        with fs.open(csv_path, 'w') as f:
            df.to_csv(f, index=False)
        print(f"\n💾 Sauvegardé sur S3: {csv_path}")
        
        json_path = f"{results_path}processing_summary.json"
        with fs.open(json_path, 'w') as f:
            json.dump(processing_summary, f, indent=2, default=str)
        print(f"💾 Résumé sur S3: {json_path}")
        
        return df, processing_summary
        
    except Exception as e:
        print(f"❌ Erreur globale: {e}")
        return pd.DataFrame(), {}

# Test simple pour vérifier la conversion
def test_pytorch_conversion():
    """Test de conversion PyTorch"""
    
    bucket_name = os.environ.get('AWS_S3_BUCKET', 'antoinelesauvage')
    fs = s3fs.S3FileSystem()
    
    s3_path = f"s3://{bucket_name}/vergers-france/preds/0_best_fold_5.npy"
    
    print("🔍 TEST CONVERSION PYTORCH")
    
    with fs.open(s3_path, 'rb') as f:
        data = np.load(f, allow_pickle=True).item()
    
    # Tester la conversion du premier masque
    first_mask = data['instance_masks'][0]
    print(f"Masque original: {type(first_mask)}")
    
    if hasattr(first_mask, 'cpu'):
        converted_mask = first_mask.cpu().numpy()
        print(f"Masque converti: {type(converted_mask)}, shape: {converted_mask.shape}")
        print(f"Somme du masque: {converted_mask.sum()}")
    
    instances = extract_instances_properties_corrected_robust(data, "0")
    print(f"✅ Instances extraites: {len(instances)}")
    
    return instances

In [2]:
def analyze_final_results(df):
    """Analyse finale des résultats"""
    
    if df is None or len(df) == 0:
        print("❌ Aucune donnée à analyser")
        return
    
    print(f"📊 ANALYSE FINALE")
    print("=" * 50)
    
    # Statistiques globales
    print(f"🔢 Instances totales: {len(df)}")
    print(f"🗺️  Tuiles: {df['tile_id'].nunique()}")
    print(f"🏷️  Classes: {df['class_id'].nunique()}")
    print(f"📐 Surface totale: {df['area_ha'].sum():.2f} ha")
    print(f"📈 Confiance moyenne: {df['confidence'].mean():.3f}")
    
    # Top 5 des classes par nombre d'instances
    print(f"\n🏆 TOP 5 CLASSES (par nombre d'instances):")
    top_classes = df['class_id'].value_counts().head()
    for class_id, count in top_classes.items():
        class_data = df[df['class_id'] == class_id]
        total_area = class_data['area_ha'].sum()
        avg_area = class_data['area_ha'].mean()
        print(f"  Classe {class_id}: {count} instances, {total_area:.2f} ha total, {avg_area:.4f} ha moyenne")
    
    # Distribution des tailles
    print(f"\n📏 DISTRIBUTION DES TAILLES:")
    size_ranges = [
        (0, 0.1, "Très petites (<0.1 ha)"),
        (0.1, 0.5, "Petites (0.1-0.5 ha)"),
        (0.5, 1, "Moyennes (0.5-1 ha)"),
        (1, 5, "Grandes (1-5 ha)"),
        (5, float('inf'), "Très grandes (>5 ha)")
    ]
    
    for min_size, max_size, label in size_ranges:
        if max_size == float('inf'):
            subset = df[df['area_ha'] >= min_size]
        else:
            subset = df[(df['area_ha'] >= min_size) & (df['area_ha'] < max_size)]
        
        count = len(subset)
        if count > 0:
            total_area = subset['area_ha'].sum()
            pct = (count / len(df)) * 100
            print(f"  {label}: {count} instances ({pct:.1f}%), {total_area:.2f} ha")


In [3]:
import geopandas as gpd
import json
from shapely.geometry import Point, Polygon
import pandas as pd

# Noms des classes en français
LABEL_NAMES = [
    "Arrière-plan",
    "Prairie",
    "Blé tendre d'hiver",
    "Maïs",
    "Orge d'hiver",
    "Colza d'hiver",
    "Orge de printemps",
    "Tournesol",
    "Vigne",
    "Betterave",
    "Triticale d'hiver",
    "Blé dur d'hiver",
    "Fruits, légumes, fleurs",
    "Pommes de terre",
    "Légumineuses fourragères",
    "Soja",
    "Verger",
    "Céréales mélangées",
    "Sorgho",
    "Étiquette vide"
]

def load_tile_grid(geojson_file="grid_indre_loire.geojson"):
    """Charge la grille des tuiles depuis le GeoJSON"""
    
    print("🗺️  CHARGEMENT DE LA GRILLE DES TUILES")
    print("=" * 50)
    
    # Charger avec geopandas
    gdf = gpd.read_file(geojson_file)
    
    print(f"📊 Grille chargée:")
    print(f"  Nombre de tuiles: {len(gdf)}")
    print(f"  CRS: {gdf.crs}")
    print(f"  Étendue: {gdf.total_bounds}")
    
    # Ajouter un ID pour chaque tuile basé sur l'index
    gdf['tile_id'] = gdf.index.astype(str)
    
    # Calculer le centre de chaque tuile
    gdf['tile_center'] = gdf.geometry.centroid
    gdf['tile_center_lon'] = gdf['tile_center'].x
    gdf['tile_center_lat'] = gdf['tile_center'].y
    
    # Calculer l'étendue de chaque tuile
    bounds = gdf.bounds
    gdf['tile_minx'] = bounds['minx']
    gdf['tile_miny'] = bounds['miny']
    gdf['tile_maxx'] = bounds['maxx']
    gdf['tile_maxy'] = bounds['maxy']
    
    print(f"  Tuiles avec coordonnées:")
    for idx, row in gdf.head().iterrows():
        print(f"    Tuile {row['tile_id']}: centre ({row['tile_center_lon']:.6f}, {row['tile_center_lat']:.6f})")
    
    return gdf

def get_tile_transform_matrix(tile_geom, image_size=(128, 128)):
    """Calcule la matrice de transformation pixel -> géographique pour une tuile"""
    
    # Obtenir les limites de la tuile
    minx, miny, maxx, maxy = tile_geom.bounds
    
    # Calculer la résolution
    width_geo = maxx - minx
    height_geo = maxy - miny
    
    pixel_width = width_geo / image_size[0]
    pixel_height = height_geo / image_size[1]
    
    # Matrice de transformation affine
    # [pixel_x, pixel_y] -> [geo_x, geo_y]
    transform = {
        'minx': minx,
        'maxy': maxy,  # Coin supérieur gauche
        'pixel_width': pixel_width,
        'pixel_height': -pixel_height,  # Négatif car Y vers le bas en image
        'crs': 'EPSG:4326'  # WGS84 lat/lon
    }
    
    return transform

# Charger la grille
tile_grid = load_tile_grid("grid_indre_loire_128.geojson")

🗺️  CHARGEMENT DE LA GRILLE DES TUILES
📊 Grille chargée:
  Nombre de tuiles: 14375
  CRS: EPSG:4326
  Étendue: [ 0.05297    46.73672     1.3752901  47.71278891]
  Tuiles avec coordonnées:
    Tuile 0: centre (0.058719, 46.740660)
    Tuile 1: centre (0.058719, 46.748539)
    Tuile 2: centre (0.058719, 46.756417)
    Tuile 3: centre (0.058719, 46.764294)
    Tuile 4: centre (0.058719, 46.772170)


/tmp/ipykernel_3671/584414981.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['tile_center'] = gdf.geometry.centroid


In [4]:
def add_geographic_info_and_class_names(instances_df, tile_grid_gdf):
    """Ajoute les informations géographiques et les noms de classes"""
    
    print("🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES")
    print("=" * 60)
    
    # Créer une copie du DataFrame
    df_enhanced = instances_df.copy()
    print("AL test", df_enhanced)
    # Ajouter les noms de classes
    df_enhanced['class_name'] = df_enhanced['class_id'].apply(
        lambda x: LABEL_NAMES[x] if 0 <= x < len(LABEL_NAMES) else f"Classe {x}"
    )
    
    # Ajouter les colonnes géographiques
    geo_columns = [
        'centroid_lon', 'centroid_lat', 
        'bbox_lon_min', 'bbox_lat_min', 'bbox_lon_max', 'bbox_lat_max',
        'tile_center_lon', 'tile_center_lat'
    ]
    
    for col in geo_columns:
        df_enhanced[col] = None
    
    # Traiter chaque tuile
    processed_tiles = 0
    
    for tile_id in df_enhanced['tile_id'].unique():
        try:
            # Trouver la géométrie de la tuile
            tile_row = tile_grid_gdf[tile_grid_gdf['tile_id'] == tile_id]
            
            if len(tile_row) == 0:
                print(f"  ⚠️  Tuile {tile_id} non trouvée dans la grille")
                continue
            
            tile_geom = tile_row.geometry.iloc[0]
            tile_info = tile_row.iloc[0]
            
            # Calculer la transformation
            transform = get_tile_transform_matrix(tile_geom)
            
            # Sélectionner les instances de cette tuile
            tile_mask = df_enhanced['tile_id'] == tile_id
            tile_instances = df_enhanced[tile_mask]
            
            print(f"  🔄 Tuile {tile_id}: {len(tile_instances)} instances")
            
            # Convertir les coordonnées
            for idx in tile_instances.index:
                # Coordonnées du centroïde
                pixel_x = df_enhanced.loc[idx, 'centroid_x']
                pixel_y = df_enhanced.loc[idx, 'centroid_y']
                
                # Transformation pixel -> géographique
                geo_x = transform['minx'] + pixel_x * transform['pixel_width']
                geo_y = transform['maxy'] + pixel_y * transform['pixel_height']
                
                df_enhanced.loc[idx, 'centroid_lon'] = geo_x
                df_enhanced.loc[idx, 'centroid_lat'] = geo_y
                
                # Boîte englobante
                bbox_x = df_enhanced.loc[idx, 'bbox_x1']
                bbox_y = df_enhanced.loc[idx, 'bbox_y1']
                bbox_w = df_enhanced.loc[idx, 'bbox_x2']
                bbox_h = df_enhanced.loc[idx, 'bbox_y2']
                
                bbox_lon_min = transform['minx'] + bbox_x1 * transform['pixel_width']
                bbox_lat_max = transform['maxy'] + bbox_y1 * transform['pixel_height']
                bbox_lon_max = transform['minx'] + (bbox_x1 + bbox_w) * transform['pixel_width']
                bbox_lat_min = transform['maxy'] + (bbox_y + bbox_h) * transform['pixel_height']
                
                df_enhanced.loc[idx, 'bbox_lon_min'] = bbox_lon_min
                df_enhanced.loc[idx, 'bbox_lat_min'] = bbox_lat_min
                df_enhanced.loc[idx, 'bbox_lon_max'] = bbox_lon_max
                df_enhanced.loc[idx, 'bbox_lat_max'] = bbox_lat_max
                
                # Informations de la tuile
                df_enhanced.loc[idx, 'tile_center_lon'] = tile_info['tile_center_lon']
                df_enhanced.loc[idx, 'tile_center_lat'] = tile_info['tile_center_lat']
            
            processed_tiles += 1
            
        except Exception as e:
            print(f"  ❌ Erreur pour la tuile {tile_id}: {e}")
            continue
    
    print(f"✅ {processed_tiles} tuiles traitées avec succès")
    
    # Statistiques par classe avec noms
    print(f"\n📊 DISTRIBUTION PAR CLASSE (avec noms):")
    class_stats = df_enhanced.groupby(['class_id', 'class_name']).agg({
        'instance_id': 'count',
        'area_ha': ['sum', 'mean'],
        'confidence': 'mean'
    }).round(4)
    
    class_stats.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in class_stats.columns]
    print(class_stats)
    
    return df_enhanced

def convert_polygon_coordinates_to_geographic(instances_df, tile_grid_gdf):
    """Convertit les coordonnées des polygones en géographique"""
    
    print("🔄 CONVERSION DES POLYGONES EN COORDONNÉES GÉOGRAPHIQUES")
    print("=" * 50)
    
    enhanced_instances = []
    
    for idx, instance in instances_df.iterrows():
        try:
            tile_id = instance['tile_id']
            
            # Trouver la tuile correspondante
            tile_row = tile_grid_gdf[tile_grid_gdf['tile_id'] == tile_id]
            if len(tile_row) == 0:
                continue
                
            tile_geom = tile_row.geometry.iloc[0]
            transform = get_tile_transform_matrix(tile_geom)
            
            # Convertir les coordonnées du polygone
            pixel_coords = instance['polygon_coords']
            geo_coords = []
            
            for pixel_coord in pixel_coords:
                px, py = pixel_coord
                geo_x = transform['minx'] + px * transform['pixel_width']
                geo_y = transform['maxy'] + py * transform['pixel_height']
                geo_coords.append([geo_x, geo_y])
            
            # Créer une nouvelle instance avec coordonnées géographiques
            enhanced_instance = instance.to_dict()
            enhanced_instance['polygon_coords_geo'] = geo_coords
            enhanced_instances.append(enhanced_instance)
            
        except Exception as e:
            print(f"  ⚠️ Erreur pour l'instance {instance.get('instance_id', idx)}: {e}")
            continue
    
    print(f"✅ {len(enhanced_instances)} polygones convertis")
    return enhanced_instances

In [5]:
def create_complete_georeferenced_geojson(enhanced_instances, filename="instances_georeferenced_complete.geojson"):
    """Crée un GeoJSON complet avec toutes les informations"""
    
    print("🗺️  CRÉATION DU GEOJSON GÉORÉFÉRENCÉ COMPLET")
    print("=" * 50)
    
    features = []
    
    for instance in enhanced_instances:
        try:
            if 'polygon_coords_geo' not in instance:
                continue
                
            coords = instance['polygon_coords_geo']
            
            # Fermer le polygone si nécessaire
            if len(coords) > 0 and coords[0] != coords[-1]:
                coords.append(coords[0])
            
            # Créer la feature
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [coords]
                },
                "properties": {
                    # Identifiants
                    "tile_id": instance['tile_id'],
                    "instance_id": instance['instance_id'],
                    
                    # Classification
                    "class_id": int(instance['class_id']),
                    "class_name": instance.get('class_name', LABEL_NAMES[instance['class_id']] if instance['class_id'] < len(LABEL_NAMES) else f"Classe {instance['class_id']}"),
                    "confidence": round(float(instance['confidence']), 3),
                    "class_probability": round(float(instance.get('class_probability', 0)), 3),
                    
                    # Géométrie et surface
                    "area_ha": round(float(instance['area_ha']), 4),
                    "area_m2": round(float(instance['area_m2']), 1),
                    "perimeter_m": round(float(instance['perimeter_m']), 1),
                    
                    # Position géographique
                    "centroid_lon": round(float(instance.get('centroid_lon', 0)), 6),
                    "centroid_lat": round(float(instance.get('centroid_lat', 0)), 6),
                    
                    # Forme
                    "compactness": round(float(instance['compactness']), 3),
                    "aspect_ratio": round(float(instance['aspect_ratio']), 2),
                    "convexity": round(float(instance['convexity']), 3),
                    
                    # Informations de la tuile
                    "tile_center_lon": round(float(instance.get('tile_center_lon', 0)), 6),
                    "tile_center_lat": round(float(instance.get('tile_center_lat', 0)), 6)
                }
            }
            
            features.append(feature)
            
        except Exception as e:
            print(f"  ⚠️ Erreur pour l'instance {instance.get('instance_id', 'unknown')}: {e}")
            continue
    
    # Créer le GeoJSON avec CRS
    geojson = {
        "type": "FeatureCollection",
        "crs": {
            "type": "name",
            "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}
        },
        "features": features
    }
    
    # Sauvegarder
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(geojson, f, indent=2, ensure_ascii=False)
    
    print(f"💾 GeoJSON créé: {filename}")
    print(f"📊 {len(features)} polygones géoréférencés")
    print(f"🌍 Système de coordonnées: WGS84 (EPSG:4326)")
    
    return geojson

# Créer le GeoJSON complet
if 'enhanced_instances' in locals():
    complete_geojson = create_complete_georeferenced_geojson(enhanced_instances)

In [6]:
def analyze_by_french_class_names(df_enhanced):
    """Analyse détaillée par classe avec noms français"""
    
    print("📊 ANALYSE PAR CLASSE (NOMS FRANÇAIS)")
    print("=" * 60)
    
    # Statistiques générales
    print(f"🔢 Total: {len(df_enhanced)} instances")
    print(f"🏷️  Classes détectées: {df_enhanced['class_id'].nunique()}")
    print(f"📐 Surface totale: {df_enhanced['area_ha'].sum():.2f} ha")
    
    # Analyse détaillée par classe
    class_analysis = df_enhanced.groupby(['class_id', 'class_name']).agg({
        'instance_id': 'count',
        'area_ha': ['sum', 'mean', 'std', 'min', 'max'],
        'confidence': ['mean', 'std'],
        'compactness': 'mean'
    }).round(4)
    
    print(f"\n📋 DÉTAIL PAR CLASSE:")
    print("=" * 80)
    
    for (class_id, class_name), group in df_enhanced.groupby(['class_id', 'class_name']):
        count = len(group)
        total_area = group['area_ha'].sum()
        avg_area = group['area_ha'].mean()
        avg_conf = group['confidence'].mean()
        
        print(f"\n🏷️  {class_name} (Classe {class_id}):")
        print(f"   📊 {count} instances ({count/len(df_enhanced)*100:.1f}%)")
        print(f"   📐 Surface: {total_area:.2f} ha total, {avg_area:.4f} ha moyenne")
        print(f"   📈 Confiance moyenne: {avg_conf:.3f}")
        
        if count > 0:
            print(f"   📏 Tailles: {group['area_ha'].min():.4f} - {group['area_ha'].max():.4f} ha")
    
    # Top 5 des classes par surface
    print(f"\n🏆 TOP 5 PAR SURFACE TOTALE:")
    top_surface = df_enhanced.groupby('class_name')['area_ha'].sum().sort_values(ascending=False).head()
    for class_name, total_area in top_surface.items():
        pct = (total_area / df_enhanced['area_ha'].sum()) * 100
        print(f"   {class_name}: {total_area:.2f} ha ({pct:.1f}%)")
    
    # Top 5 des classes par nombre d'instances
    print(f"\n🔢 TOP 5 PAR NOMBRE D'INSTANCES:")
    top_count = df_enhanced['class_name'].value_counts().head()
    for class_name, count in top_count.items():
        pct = (count / len(df_enhanced)) * 100
        print(f"   {class_name}: {count} instances ({pct:.1f}%)")
    
    # Sauvegarder l'analyse
    df_enhanced.to_csv('instances_complete_with_geography_and_classes.csv', index=False)
    print(f"\n💾 Sauvegardé: instances_complete_with_geography_and_classes.csv")
    
    return df_enhanced

# Analyser si on a les données
if 'df_enhanced' in locals():
    final_analysis = analyze_by_french_class_names(df_enhanced)

In [7]:

from skimage import measure
from scipy import ndimage
def add_geographic_info_and_class_names(instances_df, tile_grid_gdf):
    """Ajoute les informations géographiques et les noms de classes"""
    
    print("🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES")
    print("=" * 60)
    
    # Créer une copie du DataFrame
    df_enhanced = instances_df.copy()
    
    # S'assurer que tile_id est en string pour les deux DataFrames
    df_enhanced['tile_id'] = df_enhanced['tile_id'].astype(str)
    tile_grid_gdf['tile_id'] = tile_grid_gdf['tile_id'].astype(str)
    
    # Ajouter les noms de classes
    df_enhanced['class_name'] = df_enhanced['class_id'].apply(
        lambda x: LABEL_NAMES[x] if 0 <= x < len(LABEL_NAMES) else f"Classe {x}"
    )
    
    # Ajouter les colonnes géographiques
    geo_columns = [
        'centroid_lon', 'centroid_lat', 
        'bbox_lon_min', 'bbox_lat_min', 'bbox_lon_max', 'bbox_lat_max',
        'tile_center_lon', 'tile_center_lat'
    ]
    
    for col in geo_columns:
        df_enhanced[col] = None
    
    # Traiter chaque tuile
    processed_tiles = 0
    
    for tile_id in df_enhanced['tile_id'].unique():
        try:
            # Trouver la géométrie de la tuile
            tile_row = tile_grid_gdf[tile_grid_gdf['tile_id'] == tile_id]
            
            if len(tile_row) == 0:
                print(f"  ⚠️  Tuile {tile_id} non trouvée dans la grille")
                continue
            
            tile_geom = tile_row.geometry.iloc[0]
            tile_info = tile_row.iloc[0]
            
            # Calculer la transformation
            transform = get_tile_transform_matrix(tile_geom)
            
            # Sélectionner les instances de cette tuile
            tile_mask = df_enhanced['tile_id'] == tile_id
            tile_instances_indices = df_enhanced[tile_mask].index
            
            print(f"  🔄 Tuile {tile_id}: {len(tile_instances_indices)} instances")
            
            # Convertir les coordonnées pour chaque instance
            for idx in tile_instances_indices:
                # Coordonnées du centroïde
                pixel_x = df_enhanced.loc[idx, 'centroid_x']
                pixel_y = df_enhanced.loc[idx, 'centroid_y']
                
                # Transformation pixel -> géographique
                geo_x = transform['minx'] + pixel_x * transform['pixel_width']
                geo_y = transform['maxy'] + pixel_y * transform['pixel_height']
                
                df_enhanced.loc[idx, 'centroid_lon'] = geo_x
                df_enhanced.loc[idx, 'centroid_lat'] = geo_y
                
                # Boîte englobante - CORRECTION ICI
                bbox_x1 = df_enhanced.loc[idx, 'bbox_x1']
                bbox_y1 = df_enhanced.loc[idx, 'bbox_y1']
                bbox_x2 = df_enhanced.loc[idx, 'bbox_x2']
                bbox_y2 = df_enhanced.loc[idx, 'bbox_y2']
                
                bbox_lon_min = transform['minx'] + bbox_x1 * transform['pixel_width']
                bbox_lat_max = transform['maxy'] + bbox_y1 * transform['pixel_height']
                bbox_lon_max = transform['minx'] + bbox_x2 * transform['pixel_width']
                bbox_lat_min = transform['maxy'] + bbox_y2 * transform['pixel_height']
                
                df_enhanced.loc[idx, 'bbox_lon_min'] = bbox_lon_min
                df_enhanced.loc[idx, 'bbox_lat_min'] = bbox_lat_min
                df_enhanced.loc[idx, 'bbox_lon_max'] = bbox_lon_max
                df_enhanced.loc[idx, 'bbox_lat_max'] = bbox_lat_max
                
                # Informations de la tuile
                df_enhanced.loc[idx, 'tile_center_lon'] = tile_info['tile_center_lon']
                df_enhanced.loc[idx, 'tile_center_lat'] = tile_info['tile_center_lat']
            
            processed_tiles += 1
            
        except Exception as e:
            print(f"  ❌ Erreur pour la tuile {tile_id}: {e}")
            continue
    
    print(f"✅ {processed_tiles} tuiles traitées avec succès")
    
    return df_enhanced
    
def extract_polygon_from_mask(mask):
    """Extrait les coordonnées du polygone depuis un masque binaire"""
    try:
        # Trouver les contours
        contours = measure.find_contours(mask, 0.5)
        
        if len(contours) == 0:
            return []
        
        # Prendre le contour le plus long (le principal)
        main_contour = max(contours, key=len)
        
        # Simplifier le contour si trop de points
        if len(main_contour) > 100:
            # Sous-échantillonner
            step = len(main_contour) // 50
            main_contour = main_contour[::step]
        
        # Convertir en liste de coordonnées [x, y]
        polygon_coords = [[float(point[1]), float(point[0])] for point in main_contour]
        
        return polygon_coords
        
    except Exception as e:
        print(f"Erreur extraction polygone: {e}")
        return []


def extract_instances_properties_with_polygons(data, tile_id):
    """Version qui extrait aussi les polygones des masques"""
    
    instances = []
    
    try:
        # Vérifier la présence des clés essentielles
        required_keys = ['instance_masks', 'instance_boxes', 'semantic']
        missing_keys = [key for key in required_keys if key not in data]
        if missing_keys:
            print(f"    ⚠️ Clés manquantes: {missing_keys}")
            return instances
        
        instance_masks = data['instance_masks']
        instance_boxes = data['instance_boxes']
        semantic_preds = data['semantic']
        
        # Vérifier qu'il y a des instances
        if len(instance_masks) == 0:
            print(f"    ⚠️ Aucun masque d'instance trouvé")
            return instances
        
        # Gérer confidence
        if isinstance(data['confidence'], (int, float)):
            confidences = [data['confidence']] * len(instance_masks)
        elif isinstance(data['confidence'], np.ndarray):
            confidences = data['confidence'].flatten()
        elif hasattr(data['confidence'], 'cpu'):
            confidences = data['confidence'].cpu().numpy().flatten()
        else:
            confidences = [0.5] * len(instance_masks)
        
        # Convertir instance_boxes et semantic_preds si nécessaire
        if hasattr(instance_boxes, 'cpu'):
            instance_boxes = instance_boxes.cpu().numpy()
        if hasattr(semantic_preds, 'cpu'):
            semantic_preds = semantic_preds.cpu().numpy()
        
        print(f"    📊 Traitement de {len(instance_masks)} instances...")
        
        valid_instances = 0
        for i, (mask, bbox) in enumerate(zip(instance_masks, instance_boxes)):
            try:
                # Convertir le masque
                if hasattr(mask, 'cpu'):
                    mask = mask.cpu().numpy()
                elif not isinstance(mask, np.ndarray):
                    continue
                
                # Vérifier que le masque n'est pas vide
                if mask.sum() == 0:
                    continue
                
                # Extraire le polygone depuis le masque
                polygon_coords = extract_polygon_from_mask(mask)
                if not polygon_coords:
                    continue
                
                # Calculer l'aire en pixels
                area_pixels = int(mask.sum())
                area_ha = area_pixels * 0.01  # Conversion approximative
                
                # Classe prédite
                if i < len(semantic_preds):
                    class_probs = semantic_preds[i]
                    if isinstance(class_probs, np.ndarray) and len(class_probs) > 0:
                        class_id = int(np.argmax(class_probs))
                        class_confidence = float(np.max(class_probs))
                    else:
                        class_id = 0
                        class_confidence = 0.5
                else:
                    class_id = 0
                    class_confidence = 0.5
                
                # Informations géométriques
                y1, x1, y2, x2 = bbox
                centroid_x = float((x1 + x2) / 2)
                centroid_y = float((y1 + y2) / 2)
                
                instance = {
                    'tile_id': tile_id,
                    'instance_id': i,
                    'class_id': class_id,
                    'confidence': float(confidences[i]) if i < len(confidences) else 0.5,
                    'class_confidence': class_confidence,
                    'area_pixels': area_pixels,
                    'area_ha': area_ha,
                    'centroid_x': centroid_x,
                    'centroid_y': centroid_y,
                    'bbox_x1': float(x1),
                    'bbox_y1': float(y1),
                    'bbox_x2': float(x2),
                    'bbox_y2': float(y2),
                    'polygon_coords': polygon_coords  # Ajout des coordonnées du polygone
                }
                
                instances.append(instance)
                valid_instances += 1
                    
            except Exception as e:
                print(f"    ❌ Erreur instance {i}: {e}")
                continue
        
        print(f"    ✅ {valid_instances} instances valides extraites")
                
    except Exception as e:
        print(f"    ❌ Erreur globale: {e}")
    
    return instances

In [ ]:
def complete_processing_pipeline_corrected():
    """Pipeline complet corrigé"""
    
    print("🚀 PIPELINE COMPLET DE TRAITEMENT (CORRIGÉ)")
    print("=" * 60)
    
    try:
        # 1. Charger la grille des tuiles
        print("📍 Chargement de la grille...")
        tile_grid = load_tile_grid("grid_indre_loire_128.geojson")
        
        # 2. Traiter les instances avec polygones
        print("📊 Traitement des instances...")
        df_all, summary = process_all_files_safe_s3fs_pytorch()
        
        if df_all is None or len(df_all) == 0:
            print("❌ Aucune instance trouvée")
            return None
        
        print(f"✅ {len(df_all)} instances trouvées")
        
        # 3. Ajouter les informations géographiques
        print("🌍 Ajout des informations géographiques...")
        df_enhanced = add_geographic_info_and_class_names(df_all, tile_grid)
        
        # 4. Sauvegarder les résultats
        csv_filename = 'instances_georeferenced_complete.csv'
        df_enhanced.to_csv(csv_filename, index=False)
        print(f"💾 Résultats sauvegardés: {csv_filename}")
        
        # 5. Analyse finale
        print("📊 Analyse finale...")
        analyze_by_french_class_names(df_enhanced)
        
        return df_enhanced
        
    except Exception as e:
        print(f"❌ Erreur dans le pipeline: {e}")
        import traceback
        traceback.print_exc()
        return None

# Lancer le pipeline corrigé
print("🎯 LANCEMENT DU PIPELINE CORRIGÉ")
results = complete_processing_pipeline_corrected()

🎯 LANCEMENT DU PIPELINE CORRIGÉ
🚀 PIPELINE COMPLET DE TRAITEMENT (CORRIGÉ)
📍 Chargement de la grille...
🗺️  CHARGEMENT DE LA GRILLE DES TUILES
📊 Grille chargée:
  Nombre de tuiles: 14375
  CRS: EPSG:4326
  Étendue: [ 0.05297    46.73672     1.3752901  47.71278891]


/tmp/ipykernel_5655/584414981.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['tile_center'] = gdf.geometry.centroid


  Tuiles avec coordonnées:
    Tuile 0: centre (0.058719, 46.740660)
    Tuile 1: centre (0.058719, 46.748539)
    Tuile 2: centre (0.058719, 46.756417)
    Tuile 3: centre (0.058719, 46.764294)
    Tuile 4: centre (0.058719, 46.772170)
📊 Traitement des instances...
🎯 Traitement de 14375 fichiers depuis S3...

📍 [1/14375] 0_best_fold_5.npy
    📊 Traitement de 183 instances...
    ✅ 183 instances valides extraites sur 183
  ✅ 183 instances extraites

📍 [2/14375] 10000_best_fold_5.npy
    📊 Traitement de 185 instances...
    ✅ 185 instances valides extraites sur 185
  ✅ 185 instances extraites

📍 [3/14375] 10001_best_fold_5.npy
    📊 Traitement de 208 instances...
    ✅ 208 instances valides extraites sur 208
  ✅ 208 instances extraites

📍 [4/14375] 10002_best_fold_5.npy
    📊 Traitement de 173 instances...
    ✅ 173 instances valides extraites sur 173
  ✅ 173 instances extraites

📍 [5/14375] 10003_best_fold_5.npy
    📊 Traitement de 260 instances...
    ✅ 260 instances valides extraites

In [ ]:
import numpy as np
import pandas as pd
import json
import s3fs
import os
import torch
os.environ["AWS_ACCESS_KEY_ID"] = 'JBO3WDJHSHCYGRQRQB7G'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'n5vb3YgmJL1gl2oPvLpVxMF2srp+7RqoEC51G3sK'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJKQk8zV0RKSFNIQ1lHUlFSUUI3RyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU0ODEzMTU2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImFudG9pbmUubGVzYXV2YWdlQGFncmljdWx0dXJlLmdvdXYuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU1NDE3OTY4LCJmYW1pbHlfbmFtZSI6IkxFU0FVVkFHRSIsImdpdmVuX25hbWUiOiJBbnRvaW5lIiwiZ3JvdXBzIjpbIlVTRVJfT05ZWElBIl0sImlhdCI6MTc1NDgxMzE2OCwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDowNGFhNTE3NS0yNWRkLWQ4MzUtZDE4NS1jMTMyYjJkM2JhMzEiLCJsb2NhbGUiOiJmciIsIm5hbWUiOiJBbnRvaW5lIExFU0FVVkFHRSIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhbnRvaW5lbGVzYXV2YWdlIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzaWQiOiJlZGNlNzk5YS0xNjRiLTQxNDQtYWU4NC1iMDFmNzU2MjY2ZTQiLCJzdWIiOiJlOWY4NjIyYy01MzQ2LTRjOGUtOWZmYi1jY2MwMzZjY2ZjZjciLCJ0eXAiOiJCZWFyZXIifQ.X9134-3PtMnhS8ohJCwFNHIo9t0BgNPm4yzSYcHAFPAl1Fp22D89U8P9-zK4I25BeiqgiMiHTLmJVluSSLjoaw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
def process_with_checkpoints_and_resume():
    """Version avec sauvegarde intermédiaire et reprise"""
    
    # Vérifier s'il existe déjà des résultats partiels
    checkpoint_file = "processing_checkpoint.json"
    processed_tiles_file = "processed_tiles.txt"
    
    # Charger les tuiles déjà traitées
    processed_tiles = set()
    if os.path.exists(processed_tiles_file):
        with open(processed_tiles_file, 'r') as f:
            processed_tiles = set(line.strip() for line in f)
        print(f"📂 Reprise: {len(processed_tiles)} tuiles déjà traitées")
    
    # Charger les instances déjà extraites
    all_instances = []
    if os.path.exists("instances_partial.csv"):
        df_existing = pd.read_csv("instances_partial.csv")
        all_instances = df_existing.to_dict('records')
        print(f"📂 {len(all_instances)} instances déjà extraites")
    
    bucket_name = os.environ.get('AWS_S3_BUCKET', 'antoinelesauvage')
    fs = s3fs.S3FileSystem(
        client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
        key = os.environ["AWS_ACCESS_KEY_ID"], 
        secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
        token = os.environ["AWS_SESSION_TOKEN"]
    )
    
    preds_path = f"{bucket_name}/vergers-france/preds/"
    all_files = fs.glob(f"{preds_path}*_best_fold_*.npy")
    pred_files = [f"s3://{f}" for f in all_files]
    
    print(f"🎯 {len(pred_files)} fichiers totaux, {len(processed_tiles)} déjà traités")
    
    # Filtrer les fichiers non traités
    remaining_files = []
    for s3_path in pred_files:
        filename = os.path.basename(s3_path)
        tile_id = filename.split('_')[0]
        if tile_id not in processed_tiles:
            remaining_files.append(s3_path)
    
    print(f"📋 {len(remaining_files)} fichiers restants à traiter")
    
    processing_summary = {}
    successful_files = len(processed_tiles)
    error_files = 0
    
    # Traitement avec sauvegarde tous les 100 fichiers
    for i, s3_path in enumerate(remaining_files):
        try:
            filename = os.path.basename(s3_path)
            tile_id = filename.split('_')[0]
            
            if i % 100 == 0:
                print(f"\n📍 [{i+1}/{len(remaining_files)}] {filename}")
            
            # Lire et traiter le fichier
            with fs.open(s3_path, 'rb') as f:
                data = np.load(f, allow_pickle=True).item()
            
            instances = extract_instances_properties_corrected_robust(data, tile_id)
            
            if instances:
                all_instances.extend(instances)
                processing_summary[tile_id] = {
                    'n_instances': len(instances),
                    'status': 'success'
                }
                successful_files += 1
            else:
                processing_summary[tile_id] = {
                    'n_instances': 0,
                    'status': 'no_instances'
                }
            
            # Marquer comme traité
            processed_tiles.add(tile_id)
            
            # Sauvegarder tous les 100 fichiers
            if (i + 1) % 100 == 0:
                print(f"💾 Sauvegarde intermédiaire...")
                
                # Sauvegarder les instances
                df_temp = pd.DataFrame(all_instances)
                df_temp.to_csv("instances_partial.csv", index=False)
                
                # Sauvegarder les tuiles traitées
                with open(processed_tiles_file, 'w') as f:
                    for tile in processed_tiles:
                        f.write(f"{tile}\n")
                
                print(f"  ✅ {len(all_instances)} instances sauvegardées")
                print(f"  ✅ {len(processed_tiles)} tuiles marquées comme traitées")
                
        except Exception as e:
            print(f"  ❌ Erreur {filename}: {e}")
            processing_summary[tile_id] = {
                'n_instances': 0,
                'status': 'error',
                'error': str(e)
            }
            error_files += 1
            
            # Marquer quand même comme "traité" pour éviter de reboucler
            processed_tiles.add(tile_id)
    
    # Sauvegarde finale
    print(f"\n💾 SAUVEGARDE FINALE...")
    df_final = pd.DataFrame(all_instances)
    df_final.to_csv("instances_complete.csv", index=False)
    
    with open("processing_summary_final.json", 'w') as f:
        json.dump(processing_summary, f, indent=2)
    
    print(f"🎉 TERMINÉ!")
    print(f"  Total instances: {len(all_instances)}")
    print(f"  Fichiers réussis: {successful_files}")
    print(f"  Fichiers erreurs: {error_files}")
    
    return df_final, processing_summary

# Lancer la reprise
df_results, summary = process_with_checkpoints_and_resume()

🎯 14375 fichiers totaux, 0 déjà traités
📋 14375 fichiers restants à traiter

📍 [1/14375] 0_best_fold_5.npy
    📊 Traitement de 183 instances...
    ✅ 183 instances valides extraites sur 183
    📊 Traitement de 185 instances...
    ✅ 185 instances valides extraites sur 185
    📊 Traitement de 208 instances...
    ✅ 208 instances valides extraites sur 208
    📊 Traitement de 173 instances...
    ✅ 173 instances valides extraites sur 173
    📊 Traitement de 260 instances...
    ✅ 260 instances valides extraites sur 260
    📊 Traitement de 365 instances...
    ✅ 365 instances valides extraites sur 365
    📊 Traitement de 229 instances...
    ✅ 229 instances valides extraites sur 229
    📊 Traitement de 189 instances...
    ✅ 189 instances valides extraites sur 189
    📊 Traitement de 167 instances...
    ✅ 167 instances valides extraites sur 167
    📊 Traitement de 162 instances...
    ✅ 162 instances valides extraites sur 162
    📊 Traitement de 133 instances...
    ✅ 133 instances valide

KeyboardInterrupt: 

In [9]:
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp

def process_chunk_geographic(args):
    """Traite un chunk de données pour l'info géographique"""
    chunk_df, tile_grid = args
    return add_geographic_info_and_class_names(chunk_df, tile_grid)

def continue_with_existing_data_parallel():
    """Version parallélisée avec CPU"""
    
    # Chercher les fichiers existants
    possible_files = [
        "instances_partial.csv",
        "instances_complete.csv", 
        "all_instances_complete.csv"
    ]
    
    df_existing = None
    for filename in possible_files:
        if os.path.exists(filename):
            print(f"📂 Fichier trouvé: {filename}")
            df_existing = pd.read_csv(filename)
            print(f"  {len(df_existing)} instances chargées")
            break
    
    if df_existing is None:
        print("❌ Aucun fichier de sauvegarde trouvé")
        return None
    
    print("🌍 Ajout des informations géographiques (parallélisé)...")
    
    try:
        tile_grid = load_tile_grid("grid_indre_loire_128.geojson")
        
        # Diviser en chunks
        n_cores = mp.cpu_count()
        chunk_size = len(df_existing) // n_cores
        chunks = [df_existing[i:i+chunk_size] for i in range(0, len(df_existing), chunk_size)]
        
        print(f"🔄 Traitement sur {n_cores} cœurs, {len(chunks)} chunks")
        
        # Traitement parallèle
        with ProcessPoolExecutor(max_workers=n_cores) as executor:
            chunk_args = [(chunk, tile_grid) for chunk in chunks]
            results = list(executor.map(process_chunk_geographic, chunk_args))
        
        # Recombiner les résultats
        df_enhanced = pd.concat(results, ignore_index=True)
        
        # Sauvegarder
        df_enhanced.to_csv("instances_final_with_geography.csv", index=False)
        print("✅ Traitement géographique parallélisé terminé!")
        
        return df_enhanced
        
    except Exception as e:
        print(f"❌ Erreur traitement géographique: {e}")
        return df_existing

df_result = continue_with_existing_data_parallel()

📂 Fichier trouvé: all_instances_complete.csv
  3394309 instances chargées
🌍 Ajout des informations géographiques (parallélisé)...
🗺️  CHARGEMENT DE LA GRILLE DES TUILES
📊 Grille chargée:
  Nombre de tuiles: 14375
  CRS: EPSG:4326
  Étendue: [ 0.05297    46.73672     1.3752901  47.71278891]
  Tuiles avec coordonnées:
    Tuile 0: centre (0.058719, 46.740660)
    Tuile 1: centre (0.058719, 46.748539)
    Tuile 2: centre (0.058719, 46.756417)
    Tuile 3: centre (0.058719, 46.764294)
    Tuile 4: centre (0.058719, 46.772170)
🔄 Traitement sur 104 cœurs, 105 chunks


/tmp/ipykernel_3671/584414981.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['tile_center'] = gdf.geometry.centroid


🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES
  🔄 Tuile 0: 183 instances
🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES
🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES
  🔄 Tuile 10148: 152 instances
  🔄 Tuile 10275: 20 instances🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES

  🔄 Tuile 10276: 188 instances
  🔄 Tuile 1040: 152 instances
🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES
🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES
  🔄 Tuile 10000: 185 instances  🔄 Tuile 10537: 20 instances

  🔄 Tuile 10673: 125 instances
  🔄 Tuile 10538: 312 instances
  🔄 Tuile 10149: 120 instances🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES

🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES  🔄 Tuile 10801: 125 instances

  🔄 Tuile 10410: 320 instances
  🔄 Tuile 10929: 112 instances
🌍 AJOUT DES INFORMATIONS GÉOGRAPHIQUES ET CLASSES  🔄 Tuile 10277: 222 instances

  🔄 Tuile 10674: 236 instances
  🔄 Tuile 11054: 178 instances  🔄 Tuile 1014: 174 instances

🌍 AJOUT DES INFORMATIONS GÉOGRA